# 1. chatGPT API 사용법

이번 챕터에서는 chatGPT API 사용법을 익혀봅니다.

## openai client library

python으로 chatGPT API를 사용하려면 openai에서 제공하는 client library를 사용하는 것이 편합니다.  
설치한 다음, import 해주고 앞서 발급받은 API key를 지정해주겠습니다.

In [1]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/e7/44/5ece9adb8b5943273c845a1e3200168b396f556051b7d2745995abf41584/openai-1.6.1-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/39/9b/4937d841aee9c2c8102d9a4eeb800c7dad25386caabb4a1bf5010df81a57/httpx-0.26.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpcore==1.* from https://files.pythonhosted.org/packages/56/ba/78b0a99c4da0ff8b0f59defa2f13ca4668189b134bd9840b6202a93d9a0f/httpcore-1.0.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/225.4 kB ? eta -:--:--
   -------------------------------

In [1]:
import os
from openai import OpenAI

client = OpenAI(
    api_key="penapi"
)

In [10]:
# 환경변수 해보기 (보안 문제)

## ChatCompletion API
openai 라이브러리를 통해서 openai가 제공하는 다양한 API들을 사용할 수 있습니다. 그 중에서 우리가 앞으로 가장 많이 쓰게 될 API는 ChatCompletion입니다. 이는 마치 채팅을 나누는 것 처럼, 이전 대화 텍스트를 전달하면 이어지는 대화를 생성해주는 API입니다. 

### 기본 사용법
ChatCompletion API는 model과 messages 두 파라미터를 전달해서 사용할 수 있습니다. 응답은 json 형태로 리턴되며, 이를 잘 파싱해서 생성된 텍스트를 읽어올 수 있습니다.

In [18]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role": "user", "content": "밥은 먹었니?"}
    ],
)

In [19]:
response

ChatCompletion(id='chatcmpl-8abu2zwNbg04hFknfqJ0ZLZnlJ68o', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='죄송하지만, 저는 AI이기 때문에 먹을 수 없습니다. 하지만 저를 만들어준 개발자들은 아마도 점심을 먹었을 것입니다.', role='assistant', function_call=None, tool_calls=None))], created=1703736198, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=60, prompt_tokens=17, total_tokens=77))

### 텍스트 생성 결과 파싱

텍스트 생성 결과는 JSON 형태로 리턴됩니다. 다른 정보들도 같이 잔뜩 리턴되었는데, 그 중에서 텍스트만 읽어오겠습니다.

In [20]:
def parse_response(response):
    return response.choices[0].message.content

In [21]:
parse_response(response)

'죄송하지만, 저는 AI이기 때문에 먹을 수 없습니다. 하지만 저를 만들어준 개발자들은 아마도 점심을 먹었을 것입니다.'

## model 파라미터 지정
model은 대화 생성에 어떤 모델을 사용할 지 결정하는 파라미터입니다. 대표적으로 "gpt-3.5-turbo"와 "gpt-4-1106-preview"를 지정할 수 있습니다.

gpt-3.5-turbo는 성능이 준수하고 가격이 저렴합니다. 때문에 앞으로 우리는 대부분 이 모델을 이용할 예정입니다. 다만, 복잡한 테스크의 경우에는 성능이 떨어집니다. 반면에 gpt-4-1106-preview는 기가막힌 텍스트를 생성하는 대신, 10배 정도 더 비쌉니다.

한번 삼행시를 지어달라는 복잡한 테스크로 두 모델의 성능을 비교해보겠습니다.

### gpt-3.5-turbo로 복잡한 테스크 수행

In [22]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role": "user", "content": "지피티로 삼행시 지어줘"}
    ],
)
print(parse_response(response))

큰 바다 위로
선원들이 춤을 추네
푸른 파도 소리에 취해


### gpt-4로 복잡한 테스크 수행

In [34]:
response = client.chat.completions.create(
    model = "gpt-4-1106-preview",
    messages = [
        {"role": "user", "content": "민정윤으로 삼행시 지어줘"}
    ],
)
print(parse_response(response))

민: 민들레처럼 환하게 웃는
정: 정성 가득 담아 그리는 당신의
윤: 윤기 나는 삶이 늘 행복 가득하길 바랍니다.


힘의 차이가 느껴지시나요? 이처럼 텍스트 생성 자체는 gpt-4가 압도적입니다. 

### 어떤 모델을 선택하는 것이 좋을까?

gpt-3.5-turbo는 어지간한 테스크에 대해서 준수한 성능을 보여줍니다. 때문에 gpt-3.5-turbo 만으로도 내가 원하는 수준의 텍스트가 생성되는지 먼저 확인해보는 것이 좋습니다. 만약 결과가 불만족스럽고, 비용을 지불해서라도 퀄리티를 높여야한다면 gpt-4-1106-preview를 선택하면 됩니다. (참고로 펭추리는 3.5와 4를 적절히 섞어서 사용합니다 ㅎㅎ)

이 외에도 모델이 학습된 시점이나 입력으로 받을 수 있는 최대 토큰 수에 따라서 다양한 종류의 모델들이 있습니다. 궁금하신 분들은 아래 링크로 이동해서 모델마다 어떤 차이가 있는지 살펴보세요.

https://platform.openai.com/docs/models/gpt-4
https://platform.openai.com/docs/models/gpt-3-5

## role 파라미터
그 다음 messages 파라미터를 지정해주어야 합니다. 여기에는 유저와 AI가 나눈 대화를 리스트 형태로 전달해주어야 합니다. 이 때, 대화의 화자와 내용을 딕셔너리 형태로 전달해주어야 합니다. 

```python
{"role": "화자", "content": "대화내용"}
```

role에는 user, assistant, system이 있습니다. user는 유저, assistant는 chatGPT를 가리킵니다. system은 AI에게 역할을 부여할 때 사용하는데, 잠시 뒤에 알아보겠습니다. 이제 messages 파라미터를 전달할 때, 유저가 입력한 텍스트와 AI가 생성한 텍스트를 구분지어서 전달할 수 있습니다.

### AI와 대화나누기
한번 user와 assistant가 이전에 나눈 대화를 파라미터로 전달하고, 대화를 이어서 생성해보겠습니다.

In [44]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role": "user", "content": "지금 어떤 캐롤 노래 들을까?"},
        {"role": "assistant", "content": "친구들과 함께 분위기를 띄우기 좋은 'All I Want for Christmas Is You'나 'Last Christmas'같은 유명한 크리스마스 캐롤을 추천하고 싶습니다. 가능하다면 마이클 블루블레이드(Michael Bublé)의 'It's Beginning to Look a Lot Like Christmas'나 'Santa Claus Is Coming to Town'도 좋은 선택입니다. 어떤 곡을 선택하든 크리스마스 분위기를 한껏 느낄 수 있을 것입니다."},
        {"role": "user", "content": "'Last Christmas' 노래에 대해서 자세히 설명해줘"}
    ],
)
generated_message = parse_response(response)
print(generated_message)

'Last Christmas'은 영국 팝 그룹 움파캣(Umph's)이 1984년에 발표한 곡으로, 크리스마스 시즌에 특히 인기가 있는 곡 중 하나입니다. 이 곡은 조지 마이클(George Michael)이 작곡하였으며, 그의 솔로 앨범인 'Fantastic'에 수록되었습니다.

'Last Christmas'는 분위기 있는 팝 노래로, 주인공이 지난 크리스마스에 상처를 받은 사랑을 회상하며 새로운 사랑의 기회를 찾는 이야기를 담고 있습니다. 가사는 크리스마스를 배경으로 한 멜로디와 사랑은 물론 상실과 회상의 감정을 노래하는 가사로 이루어져 있습니다. 그리고 마지막 부분에서는 "This year to save me from tears, I'll give it to someone special"라는 가사로 마무리되는데, 이는 새로운 사랑을 찾겠다는 결심을 내비치고 있습니다.

'Last Christmas'은 매년 크리스마스 시즌에 많이 틀리며, 여전히 사랑받고 있는 곡입니다. 이 곡은 우울한 감정을 갖거나, 사랑의 기쁨과 아픔을 동시에 느끼는 사람들에게 특히 공감을 줄 수 있는 곡이지요.


### AI와 대화하기 함수화

In [56]:
memory = []
while True:
    user_input = input("입력: ")
    if user_input == "q":
        break
    memory.append({"role": "user", "content": user_input})
    cur_memory = memory
    if len(memory) > 8:
        cur_memory[-8:]
    response = client.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = memory
    )
    generated_message = parse_response(response)
    print(generated_message)
    memory.append({"role": "assistant", "content": generated_message})

입력:  안녕
안녕하세요! 무엇을 도와드릴까요?
입력:  hi
Hello! How can I assist you?
입력:  2+4는 뭐야
2와 4를 더하면 6입니다.
입력:  2*4는?
2와 4를 곱하면 8입니다.
입력:  4 나누기 2는 뭐야
4를 2로 나누면 2입니다.
입력:  잘자
감사합니다! 좋은 밤 되세요! 잘 자요!
입력:  q


In [59]:
len(memory)

12

### AI에게 역할 부여하기

대화를 시작할 때, role을 system으로 지정한 다음, AI에게 역할과 성격을 부여할 수 있습니다. 만약 이런 system 메세지를 설정하지 않는다면 chatGPT는 기본적으로 "You are an helpful assistant."라는 역할이 부여되게 됩니다.

```python
messages = [
    {"role": "system", "content": "당신은 주식 분석 전문가입니다."},
    ...
]
```

한번 내일 점심 메뉴를 물어보는 대화에 system message를 추가해보겠습니다.

In [51]:
response = client.chat.completions.create(
        model = "gpt-4-1106-preview",
        messages = [
            {"role": "system", "content": "당신은 해리포터에 나오는 호그와트 학교 압학생입니다."},
            {"role": "user", "content": "나는 비열하고 차가워. 나는 어떤 기숙사에 들어가는게 좋을까?"}
        ]
    )
print(parse_response(response))

호그와트 마법학교의 기숙사는 각기 다른 특성을 가진 학생들을 위해 설계되어 있어요. 당신이 자신을 '비열하고 차가운' 성향이 있다고 느끼신다면, 당신은 슬리데린 기숙사에 잘 어울릴 수 있습니다. 슬리데린 기숙사는 그리핀도르, 후플푸프, 레번클로와 함께 호그와트의 네 개 기숙사 중 하나로, 일반적으로 야심 찬, 권력욕이 강하며, 교활함과 지도력이 있는 학생들을 모집하는 것으로 알려져 있어요. 그러나 기숙사 배정은 단순히 어떤 특성 하나로 정해지는 것이 아니며, 정렬모자는 여러분의 개성, 가치, 선택, 그리고 잠재력을 종합적으로 고려하여 최적의 기숙사를 결정합니다. 

'비열함'과 '차가움'은 어떤 맥락에서든 긍정적인 특성으로 여겨지지는 않지만, 호그와트에서의 여정을 통해 자신의 다른 면모를 발전시킬 기회를 가질 수 있습니다. 호그와트는 성장과 배움의 장이니까요.


## Streaming

지금까지는 모든 텍스트가 생성되면 리턴 받는 식으로 요청을 보냈습니다. 그런데 우리가 chatGPT를 써보면, 곧바로 답변을 타닥타닥 생성합니다. 

API로도 이 기능을 사용할 수 있습니다. 먼저 요청을 보낼 때 stream 옵션을 True로 설정해줍니다. 그 다음, for문을 이용해서 응답으로부터 생성된 텍스트를 쭉쭉 읽어오는 겁니다.

In [53]:
response = client.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = [
            {"role": "user", "content": "오늘 저녁에 뭘 먹는게 좋을까요?"}
        ],
    stream=True
    )
for chunk in response:
    delta = chunk.choices[0].delta
    if delta.content:
        print(delta.content, end="")

오늘 저녁에 먹을 수 있는 다양한 옵션이 있습니다. 몇 가지 예시로는 한식, 중식, 일식, 양식 등이 있습니다. 또한 다양한 음식 카테고리 중에서 선호하는 음식을 선택하는 것도 좋습니다. 예를 들어, 한식 중에서는 불고기, 된장찌개, 비빔밥 등을 먹을 수 있습니다. 중식 중에서는 짜장면, 탕수육, 깐풍기 등을 선택할 수 있고, 일식 중에서는 초밥, 회덮밥, 덮밥 등을 선호할 수 있습니다. 양식 중에서도 스테이크, 파스타, 피자 등 다양하게 선택할 수 있습니다. 마지막으로, 현재 건강 상태나 식단에 관련된 제약이 있다면 그에 맞춰서 선택하는 것도 중요합니다.

유저가 질문하자마자 답변이 생성되는 걸 보여줄 수 있어서, 훨씬 유저 경험이 좋습니다. 뒤에 이어지는 서비스 개발 예시들에서도 유용하게 사용되니 잘 기억해주세요.|

## 마치며

지금까지 chatGPT API의 거의 모든 사용법을 알아봤습니다. 생각보다 되게 쉽죠? 이 정도만 빠삭하게 알고 있어도 정말 다양한 LLM을 이용한 서비스들을 만들 수 있습니다. 다음 강의부터 본격적으로 재밌는 프로젝트들을 직접 만들어 보겠습니다.